In [1]:

import folium
from folium.plugins import HeatMap
import geopandas as gpd

#Creo Mappa Folium con locazione Fisciano
map = folium.Map(location=[40.769928, 14.788776], zoom_start=15) 

#Creo Layer per i punti di campionamento
VectorMap=folium.FeatureGroup(name="VectorMap")
#Necessario per assegnamento semaforico del colore ad ogni punto in base alla quantità di No2 rilevata
marker_colors = ["green", "yellow", "red"]

#Inserisco punti partendo dai dati nel file geojson
folium.GeoJson("./PuntiCampionatori_con_valore.geojson",
                    marker=folium.Circle(radius=20, fill_color="green", fill_opacity=0.4, color="black", weight=2), #Creo punti circolari 

                    tooltip=folium.GeoJsonTooltip(fields=["name", "value"]), # Testo che appare quando sorvolo punto
                    zoom_on_click=True, #Zoomma quando clicco su punto
                    style_function=lambda feature: {
                        #Codice semaforico per assegnamento colore interno ai punti
                                                    "fillColor":(   
                                                    marker_colors[0] if feature["properties"]["value"]<7 else
                                                   marker_colors[1] if feature["properties"]["value"]<14 else 
                                                    marker_colors[2]), 
                                                    "color": "black",
                                                    "weight": 2,
                                                    "dashArray": "5, 5",
    },
).add_to(VectorMap)
map.add_child(VectorMap)


#Creo layer per Heatmap
RasterMap=folium.FeatureGroup(name="RasterMap")
#Leggo file per estrapolare dati relativi alla posizione e valore 
gdf=gpd.read_file("PuntiCampionatori_con_valore.geojson")
data = []
for idx, row in gdf.iterrows():
    lat = row.geometry.y
    lon = row.geometry.x
    No2 = row["value"] 
    data.append([lat, lon, No2])
    
heatmap = HeatMap(data=data).add_to(RasterMap)
RasterMap.add_child(heatmap)
map.add_child(RasterMap)
folium.LayerControl().add_to(map)
map

